# Анализ базы данных

**Постановка цели:**
* проанализировать базу данных, связанную с крупным сервисом для чтения книг по подписке

**Постановка плана работы:**
* выгрузить данные
* выполненить поставленные задачи
* составить общий вывод

**Описание данных:**

таблица books - данные о книгах:

* book_id — идентификатор книги
* author_id — идентификатор автора
* title — название книги
* num_pages — количество страниц
* publication_date — дата публикации книги
* publisher_id — идентификатор издателя

таблица authors -  данные об авторах:

* author_id — идентификатор автора
* author — имя автора

таблица publishers - данные об издательствах:

* publisher_id — идентификатор издательства
* publisher — название издательства

таблица ratings - данные о пользовательских оценках книг:

* rating_id — идентификатор оценки
* book_id — идентификатор книги
* username — имя пользователя, оставившего оценку
* rating — оценка книги

таблица reviews - данные о пользовательских обзорах на книги:

* review_id — идентификатор обзора
* book_id — идентификатор книги
* username — имя пользователя, написавшего обзор
* text — текст обзора

**Постановка задач, которые необходимо выполнить:**
* Посчитать, сколько книг вышло после 1 января 2000 года
* Для каждой книги посчитать количество обзоров и среднюю оценку
* Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры
* Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками
* Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок

# Выгрузка данных 

In [2]:
import pandas as pd
from sqlalchemy import create_engine

In [3]:
db_config = {'user': 'praktikum_student', 
'pwd': 'Sdf4$2;d-d30pp',
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, 
'db': 'data-analyst-final-project-db'} 
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
db_config['pwd'],
db_config['host'],
db_config['port'],
db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [4]:
query = ''' SELECT *
            FROM books
            LIMIT 5
        '''
pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [5]:
query = ''' SELECT *
            FROM authors
            LIMIT 5
        '''
pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [6]:
query = ''' SELECT *
            FROM publishers
            LIMIT 5
        '''
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [7]:
query = ''' SELECT *
            FROM ratings
            LIMIT 5
        '''
pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [8]:
query = ''' SELECT *
            FROM reviews
            LIMIT 5
        '''
pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


# Выполнение заданий

## Задача 1. Посчитать, сколько книг вышло после 1 января 2000 года

In [9]:
query = '''
    SELECT COUNT(book_id)
    FROM books
    WHERE publication_date > '2000-01-01'
    '''
pd.io.sql.read_sql(query, con = engine)

,count
0,819


**Вывод:**
* значит 819 книг вышло после 1 января 2000 года

## Задача 2. Для каждой книги посчитать количество обзоров и среднюю оценку

In [10]:
query = '''
        SELECT t.title,
        COUNT(DISTINCT rev.review_id),
        AVG(rat.rating)
        FROM books AS t
        LEFT JOIN reviews AS rev ON t.book_id = rev.book_id
        LEFT JOIN ratings AS rat ON t.book_id = rat.book_id
        GROUP BY t.book_id
        ORDER BY avg DESC,
        count DESC;
        '''
pd.io.sql.read_sql(query, con = engine)

,title,count,avg
0,A Dirty Job (Grim Reaper #1),4,5.00
1,School's Out—Forever (Maximum Ride #2),3,5.00
2,Moneyball: The Art of Winning an Unfair Game,3,5.00
3,Arrows of the Queen (Heralds of Valdemar #1),2,5.00
4,Wherever You Go There You Are: Mindfulness Me...,2,5.00
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,3,2.25
996,Drowning Ruth,3,2.00
997,His Excellency: George Washington,2,2.00
998,Junky,2,2.00


**Вывод:**
* высокие показатели у книги "A Dirty Job" (Grim Reaper ): 4 обзора и средняя оценка, равная 5
* низкие показатели у книги "The Cat in the Hat and Other Dr. Seuss Favorites": 0 обзоров и средняя оценка, равная 5

## Задача 3. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры

In [11]:
query = ''' SELECT p.publisher,
                   COUNT(p.publisher_id) AS books_number  
            FROM books AS b
            LEFT OUTER JOIN publishers AS p ON b.publisher_id = p.publisher_id
            WHERE b.num_pages > 50
            GROUP BY p.publisher_id
            ORDER BY COUNT(p.publisher_id) DESC
            LIMIT 5;
        '''

pd.io.sql.read_sql(query, con = engine)

,publisher,books_number
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19


**Вывод:**
* Penguin Books	 -  издательство, которое выпустило наибольшее число книг толще 50 страниц

## Задача 4. Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками

In [12]:
query = '''
        SELECT author, AVG(rating)
        FROM authors AS a
        
        JOIN books AS b ON a.author_id = b.author_id

        JOIN ratings AS r ON b.book_id = r.book_id

        WHERE b.book_id IN (
                            SELECT book_id
                            FROM ratings 
                            GROUP BY book_id
                            HAVING COUNT(rating_id) >= 50
                            ORDER BY COUNT(rating_id) DESC
                            )
        GROUP BY author
        ORDER BY avg DESC
        LIMIT 5

        '''
pd.io.sql.read_sql(query, con = engine)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.287097
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.246914
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645


**Вывод:**
* J.K. Rowling/Mary GrandPré -  автор с самой высокой средней оценкой книг

## Задача 5. Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [13]:
query = ''' WITH  new_rating AS ( SELECT username
                                  FROM ratings 
                                  GROUP BY username
                                  HAVING COUNT(book_id) > 48
                               ),
                               
                 new_review AS ( SELECT username,
                                 COUNT(book_id) AS reviews_number
                                 FROM reviews
                                 GROUP BY username
                               )
                              
            SELECT AVG(rev.reviews_number) AS mean
            FROM new_review AS rev
            INNER JOIN new_rating AS rat ON rat.username = rev.username; 
        '''
pd.io.sql.read_sql(query, con = engine)

,mean
0,24.0


**Вывод:**
* 24,3 - среднее количество обзоров от пользователей, которые поставили больше 48 оценок

<div class="alert alert-info">Дополнительные задачи:</div>

## Дополнителная задача 1

**Выведите таблицу, которая будет содержать по году публикации:**
- количество издательств,
- количество выпущенных книг 
- сколько всего тысяч страниц было в изданных книгах

*(отобразить только те года, в которых издано более 30 книг)*

In [14]:
query = ''' SELECT EXTRACT (YEAR FROM publication_date) AS publ_year,
                COUNT(DISTINCT publisher_id) AS count_publishers,
                COUNT(book_id) AS count_books,
                (SUM(num_pages)/1000)AS num_pages_kilo
            FROM books
            GROUP BY publ_year
            HAVING COUNT(book_id) > 30
            ORDER BY publ_year,COUNT(book_id) DESC
            
        '''
pd.io.sql.read_sql(query, con = engine)

,publ_year,count_publishers,count_books,num_pages_kilo
0,1999.0,26,41,15
1,2000.0,35,38,13
2,2001.0,41,60,21
3,2002.0,62,94,38
4,2003.0,65,105,41
5,2004.0,88,124,46
6,2005.0,89,139,55
7,2006.0,109,184,68
8,2007.0,38,50,18


**Вывод:**
* самая "старая" дата : 1999 насчитывает :
    * 26 издательств
    * 41 выпущенных книг
    * 15 тыс. станиц в книгах
* самая "молодая" дата : 2006-09-26 насчитывает:
    * 38 издательств
    * 50 выпущенных книг
    * 18 тыс. станиц в книгах

## Дополнителная задача 2

* Выведите в одной таблице два числа — среднюю оценку тех книг, на которые написало отзывов более 3 человек и отдельно среднюю оценку остальных книг, сделайте выводы какой рейтинг больше

In [15]:
query = ''' WITH books_with_b3_rating AS 
              (SELECT book_id
                 FROM reviews
                 GROUP BY book_id
                 HAVING COUNT(DISTINCT review_id )>3),
            rating_b3 AS 
            (SELECT AVG(rating) 
               FROM ratings AS r INNER JOIN books_with_b3_rating AS b ON b.book_id = r.book_id),
            rating_l3 AS 
            (SELECT AVG(rating) 
               FROM ratings AS r LEFT JOIN books_with_b3_rating AS b ON b.book_id = r.book_id
               WHERE b.book_id IS NULL)
            SELECT * FROM rating_b3,rating_l3
                    
        '''

pd.io.sql.read_sql(query, con = engine)

,avg,avg
0,3.949505,3.890375


**Вывод:**
* средняя оценка книг, на которые написало отзывов более 3 человек : 3.95
* cредняя оценка остальных книг: 3.89
* наблюдаем, что средняя оценка книг, на которые написало отзывов более 3 человек, больше той оценки,cредняя оценка которых не более 3 

## Дополнителная задача 3

* Выведите топ пять пользователей по суммарному показателю написанных ревью и поставленных оценок

In [16]:
query = ''' WITH review_count AS 
              (SELECT username,count(*) count_rv
                 FROM reviews
                 GROUP BY username),
            rating_count AS 
              (SELECT username,count(*) count_rt
                 FROM ratings
                 GROUP BY username)
            
            SELECT rv.username,count_rv + count_rt AS sum_rt_rv 
            FROM review_count rv INNER JOIN rating_count rt ON rv.username = rt.username 
            ORDER BY count_rv + count_rt DESC
            LIMIT 5
                    
        '''

pd.io.sql.read_sql(query, con = engine)

,username,sum_rt_rv
0,martinadam,83
1,sfitzgerald,83
2,richard89,81
3,susan85,78
4,jennifermiller,78


**Вывод:**
* martinadam и sfitzgerald имеют в топе наибольшую сумму написанных ревью и поставленных оценок : 83
* susan85 и jennifermiller имеют в топе наименьшую сумму написанных ревью и поставленных оценок : 78

# Общий вывод

**Обобщим полученные данные:**
* значит 819 книг вышло после 1 января 2000 года
* высокие показатели у книги "A Dirty Job" (Grim Reaper ): 4 обзора и средняя оценка, равная 5
* низкие показатели у книги "The Cat in the Hat and Other Dr. Seuss Favorites": 0 обзоров и средняя оценка, равная 5
* Penguin Books	 -  издательство, которое выпустило наибольшее число книг толще 50 страниц
* J.K. Rowling/Mary GrandPré -  автор с самой высокой средней оценкой книг

# Вывод после дополнительного задания

**Обобщим полученные данные:**
* самая "старая" дата : 1999 насчитывает :
    * 26 издательств
    * 41 выпущенных книг
    * 15 тыс. станиц в книгах
* самая "молодая" дата : 2006-09-26 насчитывает:
    * 38 издательств
    * 50 выпущенных книг
    * 18 тыс. станиц в книгах
* средняя оценка книг, на которые написало отзывов более 3 человек : 3.95
* cредняя оценка остальных книг: 3.89
* наблюдаем, что средняя оценка книг, на которые написало отзывов более 3 человек, больше той оценки,cредняя оценка которых не более 3 
* martinadam и sfitzgerald имеют в топе наибольшую сумму написанных ревью и поставленных оценок : 83
* susan85 и jennifermiller имеют в топе наименьшую сумму написанных ревью и поставленных оценок : 78